In [13]:
import aocd
from aocd.models import Puzzle
day = 19
year = 2021
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [258]:
import re
import numpy as np
from collections import defaultdict
from functools import lru_cache

In [15]:
data = puzzle.input_data.splitlines()
len(data), data[:10]

(1061,
 ['--- scanner 0 ---',
  '615,499,683',
  '-748,-781,588',
  '392,-638,-524',
  '-453,892,-689',
  '-127,159,-103',
  '-782,-721,695',
  '793,523,765',
  '558,953,-776',
  '-960,-476,-523'])

In [16]:
test_data = """--- scanner 0 ---
0,2
4,1
3,3

--- scanner 1 ---
-1,-1
-5,0
-2,1""".splitlines()

In [83]:
test_data2 = """--- scanner 0 ---
-1,-1,1
-2,-2,2
-3,-3,3
-2,-3,1
5,6,-4
8,0,7""".splitlines()

In [143]:
test_data3 = """--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14""".splitlines()

In [330]:
def parse(data):
    scanners = {}
    cur = None
    beacons = []
    for line in data:
        if line == '':
            continue
        if 'scanner' in line:
            if cur is not None:
                scanners[cur] = np.array(beacons)
            cur = int(re.sub('[^0-9]', '', line))
            beacons = []
        else:
            beacons.append(list(map(int, line.strip().split(','))))
    scanners[cur] = np.array(beacons)
    return scanners

def overlap(scannerA, scannerB, minimum=3):
    beaconsA = set(tuple(p) for p in scannerA)
#     print(beaconsA)
    for beacon in scannerB[:-minimum+1]:
        deltas = scannerA - beacon
        for d in deltas:
            match = set(tuple(t) for t in  (scannerB + d))
#             print(match)
#             ovlp = len(match&beaconsA)
#             if ovlp:
#                 print(ovlp)
            if (len(match&beaconsA) >= minimum):
                return d
    return None

@lru_cache(maxsize=None)
def get_orientations_cached(idx):
    beacons = scanners[idx]
    orientations = [
        (None, (1, 1, 1)),
        (None, (-1, 1, -1)),
        ((0, 2), (-1, 1, 1)),
        ((0, 2), (1, 1, -1)),
        ((1, 2), (1, -1, 1)),
        ((1, 2), (1, 1, -1)),
    ]
    
    for j in range(6):
        new_beacons = beacons.copy()
        swap, mult = orientations[j]
        if swap is not None:
            a, b = swap
            new_beacons[:, [a, b]] = new_beacons[:, [b, a]]
        new_beacons *= mult
        for i in range(4):
            new_beacons[:, [0, 1]] = new_beacons[:, [1, 0]]
            new_beacons *= [-1, 1, 1]
            yield new_beacons, orientations[j], i

def get_orientations(beacons):
    orientations = [
        (None, (1, 1, 1)),
        (None, (-1, 1, -1)),
        ((0, 2), (-1, 1, 1)),
        ((0, 2), (1, 1, -1)),
        ((1, 2), (1, -1, 1)),
        ((1, 2), (1, 1, -1)),
    ]
    
    for j in range(6):
        new_beacons = beacons.copy()
        swap, mult = orientations[j]
        if swap is not None:
            a, b = swap
            new_beacons[:, [a, b]] = new_beacons[:, [b, a]]
        new_beacons *= mult
        for i in range(4):
            new_beacons[:, [0, 1]] = new_beacons[:, [1, 0]]
            new_beacons *= [-1, 1, 1]
            yield new_beacons, orientations[j], i
            
            
def get_distances(beacons):
    dist = []
    for i in range(1, len(beacons)//2 + (len(beacons)%2)+1):
#         print(i)
        diff = beacons - np.roll(beacons, i, axis=0)
        dist.extend(np.sum(np.square(diff), axis=1).tolist())
    return sorted(dist)

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
        
        

In [305]:
scanners = parse(test_data)

In [306]:
overlap(scanners[0], scanners[1])

array([5, 2])

In [307]:
scanners = parse(test_data2)
print(scanners[0])
for beacon in get_orientations(scanners[0]):
    print(beacon)


[[-1 -1  1]
 [-2 -2  2]
 [-3 -3  3]
 [-2 -3  1]
 [ 5  6 -4]
 [ 8  0  7]]
(array([[ 1, -1,  1],
       [ 2, -2,  2],
       [ 3, -3,  3],
       [ 3, -2,  1],
       [-6,  5, -4],
       [ 0,  8,  7]]), (None, (1, 1, 1)), 0)
(array([[ 1,  1,  1],
       [ 2,  2,  2],
       [ 3,  3,  3],
       [ 2,  3,  1],
       [-5, -6, -4],
       [-8,  0,  7]]), (None, (1, 1, 1)), 1)
(array([[-1,  1,  1],
       [-2,  2,  2],
       [-3,  3,  3],
       [-3,  2,  1],
       [ 6, -5, -4],
       [ 0, -8,  7]]), (None, (1, 1, 1)), 2)
(array([[-1, -1,  1],
       [-2, -2,  2],
       [-3, -3,  3],
       [-2, -3,  1],
       [ 5,  6, -4],
       [ 8,  0,  7]]), (None, (1, 1, 1)), 3)
(array([[ 1,  1, -1],
       [ 2,  2, -2],
       [ 3,  3, -3],
       [ 3,  2, -1],
       [-6, -5,  4],
       [ 0, -8, -7]]), (None, (-1, 1, -1)), 0)
(array([[-1,  1, -1],
       [-2,  2, -2],
       [-3,  3, -3],
       [-2,  3, -1],
       [ 5, -6,  4],
       [ 8,  0, -7]]), (None, (-1, 1, -1)), 1)
(array([[-1, -1, 

In [325]:
scanners[0]

array([[-1, -1,  1],
       [-2, -2,  2],
       [-3, -3,  3],
       [-2, -3,  1],
       [ 5,  6, -4],
       [ 8,  0,  7]])

In [250]:
%%time
nmatch = 12
scanners = parse(test_data3)
idx = list(scanners.keys())
nscanners = len(idx)
nscanners = 2
translations = {}

for i in range(nscanners):
    for j in range(i+1, nscanners):
        for rotated, orientation, numrot in get_orientations(scanners[j]):
#             print(i, j)
            delta = overlap(scanners[i], rotated, minimum=nmatch)
            if delta is not None:
                translations[(i, j)] = (delta, orientation, numrot)
                

CPU times: user 1.2 s, sys: 3 µs, total: 1.2 s
Wall time: 1.2 s


In [251]:
translations

{(0, 1): (array([   68, -1246,   -43]), (None, (-1, 1, -1)), 3)}

In [339]:
%%time
scanners = parse(data)

distances = {}
for key in scanners:
    distances[key] = get_distances(scanners[key])

CPU times: user 63.7 ms, sys: 11.4 ms, total: 75.2 ms
Wall time: 60.9 ms


In [341]:
%%time
nmatch = 12
# scanners = parse(test_data3)
scanners = parse(data)
translations = {}
checked = defaultdict(int)
rotations = defaultdict(list)

distances = {}
for key in scanners:
    distances[key] = get_distances(scanners[key])
    
added = []

while True:
    idx = sorted(list(scanners.keys()))
    nscanners = len(idx)
    if nscanners <= 1:
        break
    
#     print(idx)
    for j in idx[1:]:
        index = j
        print(j, index)
        if len(intersection(distances[0], distances[index])) < nmatch:
            continue
    
        if len(rotations[index]):
            
            for rotated in rotations[index]:
                delta = overlap(scanners[0], rotated, minimum=nmatch)
                if delta is not None:
                    translations[(i, index)] = (delta, orientation, numrot)
                    print("adding {}".format(index))
                    tmp = np.vstack([scanners[0], rotated+delta])
                    _, indexes = np.unique(tmp, axis=0, return_index=True)
                    scanners[0] = tmp[np.sort(indexes)]
                    distances[0] = sorted(distances[0] + distances[index])
                    scanners.pop(index)
                    break
        else:
            for rotated, orientation, numrot in get_orientations(scanners[index]):
                rotations[index].append(rotated.copy())
    #             print(i, j)
                delta = overlap(scanners[0], rotated, minimum=nmatch)
                if delta is not None:
                    translations[(i, index)] = (delta, orientation, numrot)
                    print("adding {}".format(index))
                    tmp = np.vstack([scanners[0], rotated+delta])
                    _, indexes = np.unique(tmp, axis=0, return_index=True)
                    scanners[0] = tmp[np.sort(indexes)]
                    distances[0] = sorted(distances[0] + distances[index])
                    scanners.pop(index)
                    break
        checked[index] = len(scanners[0])

1 1
2 2
adding 2
3 3
4 4
5 5
6 6
7 7
adding 7
8 8
adding 8
9 9
adding 9
10 10
11 11
adding 11
12 12
13 13
14 14
15 15
adding 15
16 16
adding 16
17 17
18 18
adding 18
19 19
20 20
21 21
adding 21
22 22
23 23
adding 23
24 24
25 25
26 26
adding 26
27 27
adding 27
28 28
29 29
adding 29
30 30
adding 30
31 31
adding 31
32 32
adding 32
33 33
34 34
35 35
adding 35
36 36
37 37
1 1
3 3
adding 3
4 4
5 5
adding 5
6 6
10 10
adding 10
12 12
adding 12
13 13
14 14
17 17
adding 17
19 19
adding 19
20 20
22 22
adding 22
24 24
adding 24
25 25
adding 25
28 28
adding 28
33 33
adding 33
34 34
36 36
adding 36
37 37
adding 37
1 1
adding 1
4 4
6 6
adding 6
13 13
adding 13
14 14
20 20
adding 20
34 34
4 4
adding 4
14 14
adding 14
34 34
adding 34
CPU times: user 2min 10s, sys: 87.8 ms, total: 2min 10s
Wall time: 2min 10s


In [342]:
len(scanners[0])

491

In [233]:
puzzle.answer_a = len(scanners[0])

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [236]:
points = [val[0] for k, val in translations.items()]

In [238]:
maxdist = 0
npoints = len(points)
for i in range(npoints):
    for j in range(i+1, npoints):
        dist = np.sum(np.abs(points[j] - points[i]))
        if dist > maxdist:
            maxdist = dist

In [240]:
puzzle.answer_b = maxdist

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 19! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [268]:
list(range(20))[:-12+1]

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [267]:
list(range(20))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [303]:
diff = beacon - np.roll(beacon, 1, axis=0)
np.sum(np.square(diff), axis=1)


array([118,   3,   3,   5, 155, 166])

In [302]:
beacon - np.roll(beacon, 1, axis=0)

array([[-9, -6,  1],
       [-1,  1,  1],
       [-1,  1,  1],
       [ 1, -2,  0],
       [ 7, -5, -9],
       [ 3, 11,  6]])